In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import evaluate

In [3]:
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("ealvaradob/phishing-dataset", "texts", trust_remote_code=True)

df = dataset['train'].to_pandas()
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, test = Dataset.from_pandas(train, preserve_index=False), Dataset.from_pandas(test, preserve_index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.75k [00:00<?, ?B/s]

phishing-dataset.py:   0%|          | 0.00/3.27k [00:00<?, ?B/s]

texts.json:   0%|          | 0.00/52.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

all records are labeled as 1 (Phishing) or 0 (Benign).

Load the tokenizer and the model

In [4]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize the datasets

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)

Map:   0%|          | 0/16109 [00:00<?, ? examples/s]

Map:   0%|          | 0/4028 [00:00<?, ? examples/s]

Define the data collator

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Define the evaluation metric using 'evaluate'

In [7]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix  # <-- Add this line

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    # Calculate accuracy, precision, and recall
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, zero_division=0)
    recall = recall_score(labels, predictions, zero_division=0)

    # Calculate confusion matrix to get False Positive Rate
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "false_positive_rate": fpr
    }


Set up training arguments

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-9-e234904ebeb9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,False Positive Rate
1,0.128900,0.111592,0.972939,0.993767,0.934853,0.003610
2,0.040000,0.052856,0.989325,0.981912,0.990228,0.011231
3,0.010900,0.065854,0.990070,0.990808,0.983062,0.005616


TrainOutput(global_step=3021, training_loss=0.0599502233904902, metrics={'train_runtime': 1073.6822, 'train_samples_per_second': 45.011, 'train_steps_per_second': 2.814, 'total_flos': 1.271536797238272e+16, 'train_loss': 0.0599502233904902, 'epoch': 3.0})

In [ ]:
test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4028
})

In [ ]:
import matplotlib.pyplot as plt

# Convert accuracy and loss values to percentages
accuracies_percentage = [acc * 100 for acc in accuracies]

# Plotting Accuracy over Epochs with modified axes
plt.figure(figsize=(6, 5), dpi=300)
plt.plot(epochs, accuracies_percentage, marker='o', color='b', label="Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.title("Training Accuracy over Epochs")
plt.xticks([1, 2, 3])  # Set x-axis to show 1, 2, 3
plt.legend()
plt.savefig('acuracy.jpg')
plt.show()

# Plotting Loss over Epochs with modified axes
plt.figure(figsize=(6, 5), dpi=300)
plt.plot(epochs, losses, marker='o', color='r', label="Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (%)")
plt.title("Training Loss over Epochs")
plt.xticks([1, 2, 3])  # Set x-axis to show 1, 2, 3
plt.legend()
plt.savefig('loss.jpg')
plt.show()


In [11]:
# Choose a directory to save the model
output_dir = "./fine_tuned_roberta_model2"

# Save the model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./fine_tuned_roberta_model2/tokenizer_config.json',
 './fine_tuned_roberta_model2/special_tokens_map.json',
 './fine_tuned_roberta_model2/vocab.json',
 './fine_tuned_roberta_model2/merges.txt',
 './fine_tuned_roberta_model2/added_tokens.json')

Load the model

In [13]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Load the model and tokenizer
model = RobertaForSequenceClassification.from_pretrained(output_dir)
tokenizer = RobertaTokenizer.from_pretrained(output_dir)

# Ensure the model is in evaluation mode
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# import pandas as pd

# csv_path = "ronit_phishing_examples_4th_run.csv"  # Replace with your CSV file path
# df = pd.read_csv(csv_path)
# df.rename(columns={"Examples": "text", "Labels": "label"}, inplace=True)

# # Convert the DataFrame to a Hugging Face Dataset
# new_dataset = Dataset.from_pandas(df, preserve_index=False)


In [ ]:
# # Step 3: Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

# new_dataset = new_dataset.map(tokenize_function, batched=True)
# new_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
# import torch
# # Step 4: Make predictions
# all_preds = []
# all_labels = []

# with torch.no_grad():
#     for batch in new_dataset:
#         input_ids = batch["input_ids"].unsqueeze(0)
#         attention_mask = batch["attention_mask"].unsqueeze(0)
#         labels = batch["label"]

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1).item()

#         all_preds.append(predictions)
#         all_labels.append(labels.item())

In [14]:
# Step 4: Make predictions Version 2
import torch
import torch.nn.functional as F

test.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

all_preds = []
all_labels = []
all_confidences = []
# all_logits = []

with torch.no_grad():
    for batch in test:
        input_ids = batch["input_ids"].unsqueeze(0)
        attention_mask = batch["attention_mask"].unsqueeze(0)
        labels = batch["label"]

        # Get model outputs
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Calculate probabilities using softmax
        probs = F.softmax(logits, dim=-1)
        confidence = probs[0, 1].item()  # Confidence for the 'phishing' class (label 1)

        # Store predictions and confidence scores
        prediction = torch.argmax(probs, dim=-1).item()
        all_preds.append(prediction)
        all_labels.append(labels.item())
        all_confidences.append(confidence)
        # all_logits.append(logits.tolist()[0])

# Example: Printing some results
for i, (text, label, confidence) in enumerate(zip(test["text"], all_labels, all_confidences)):
    if i >= 10:
      break

    print(f"Example {i + 1}:")
    print(f"Text: {text}")
    print(f"True Label: {label}")
    print(f"Confidence (Phishing): {confidence:.2f}")
    # print(f"Logit Values: {logit_val}")
    print("-" * 50)

Example 1:
Text: 
Digital Dispatch Weekly NewsletterAll CNET
 The Web
Â 
Apple to expand iMac LCD display
It ain't heavy; it's my laptop
Gateway touts chic yet cheap PCs
Apple's iPod comes to Linux
Dell PCs coming to a mall near youMore CNET NewsQuintessential Player 3.4AI Picture Utility 6.2.1ICQ 2002a Build #3728Deck 3.5 (for the Mac)Dell Latitude C510/C610 seriesÂ 
In HardwareToshiba Pocket PC e310 
In ElectronicsAutoCAD LT 2002 
In SoftwareSony Ericsson T68 
In Wireless
 July 11, 2002						--Janice Chen,						editor in chief,						CNET Reviews
 Dear readers,It was a crushing blow to discover that Vindigo, my
all-time-favorite Palm app, was no longer free. Twenty-five bucks
wasn't steep, but the principle of paying for a former freebie
was hard to swallow. Not that hard, though; surprising
myself--and my fellow cheapskates--I've discovered that I'd
rather dish out the dough than go Vindigo-free myself.That's what Web-based e-mail providers are banking on. Now that
most of us have go

In [15]:
for i in range(10):
    print(all_confidences[i])

6.375795783242211e-05
5.0209804612677544e-05
5.421388414106332e-05
3.834386370726861e-05
2.9414781238301657e-05
2.9655797334271483e-05
2.928890535258688e-05
0.999946117401123
3.3703490771586075e-05
5.60299631615635e-05


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix data
cm = confusion_matrix(all_labels, all_preds)

# Customize the appearance with seaborn heatmap
plt.figure(figsize=(8, 6), dpi=300)
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    linewidths=0.5,
    linecolor='gray',
    cbar_kws={'shrink': 0.75},
    annot_kws={"size": 14}
)

# Add labels and titles for clarity
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.xticks(ticks=[0.5, 1.5], labels=['0', '1'], fontsize=12)
plt.yticks(ticks=[0.5, 1.5], labels=['0', '1'], fontsize=12, rotation=0)

plt.savefig('ConfusionMatrix.jpg')
# Display the improved confusion matrix
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(all_labels, all_preds, target_names=['Class 0', 'Class 1'])

# Print the report
print("Classification Report:")
print(report)
